### NAVER API로부터 평가요소들의 위경도 값 추출

In [23]:
import pandas as pd
import os
import json
import requests
import pyproj

In [3]:
# 빵소 (https://pangpangday.co.kr/bbs/board.php?bo_table=b302)
pan_data = [
    {'name':'꼬망스케익', 'addr':'천안 서북구 봉정로 362'}, # 31103
    {'name':'듀팡과자점', 'addr':'천안 서북구 늘푸른5길 22'},
    {'name':'뚜쥬루돌가마점', 'addr':'천안 동남구 풍세로 706'},
    {'name':'못난이꽈배기', 'addr':'천안 동남구 사직로 10-6'},
    {'name':'몽상가인', 'addr':'천안 서북구 쌍용대로 85'},  # 31155
    {'name':'브레드보드', 'addr':'천안 동남구 성남면 종합휴양지로 183'},
    {'name':'쁘띠빠리', 'addr':'천안 서북구 한들3로 88'},
    {'name':'수제빵연구소', 'addr':'천안 서북구 백석3로 35'},
    {'name':'슈에뜨베이커리', 'addr':'천안 동남구 서부대로 226-12'},
    {'name':'시바앙과자점', 'addr':'천안 동남구 충절로 42'}, # 경남아너스빌상가
    {'name':'엘림제과', 'addr':'천안 동남구 병천면 충절로 1747'},
    {'name':'주식회사 호도원본점', 'addr':'천안 동남구 만남로 126'},
    {'name':'지씨브레드', 'addr':'천안 동남구 유량로 196'},
    {'name':'천안당호두과자남천안점', 'addr':'천안 동남구 천안대로 87'}, # 목천읍
    {'name':'천안옛날호두과자본점', 'addr':'천안 동남구 광풍로 1739'},  #  1층 (구룡동)
    {'name':'파스텔베이커리', 'addr':'천안 동남구 신부4길 24-5'},
    {'name':'할머니 학화호도과자 본점', 'addr':'천안 동남구 만남로 62'},
]

pan_df = pd.DataFrame(pan_data)

In [4]:
# 폐건물
building_data = [
    {'name':'두정동 폐공사장', 'addr':'충남 천안시 서북구 두정동 1354'},
    {'name':'쌍용동 폐공장', 'addr':'천안시 서북구 충무로 214'},
    {'name':'삼룡동 폐건물', 'addr':'천안시 동남구 삼룡동 240-3'}
]

building_df = pd.DataFrame(building_data)

In [5]:
# 교통 인프라
infra_data = [
    {'name':'천안역 1호선', 'addr':'천안 동남구 대흥로 239'},
    {'name':'천안아산역', 'addr':'충청남도 아산시 배방읍 희망로 100'},
    {'name':'천안고속버스터미널', 'addr':'천안 동남구 만남로 43'},
    {'name':'두정역', 'addr':'천안 서북구 두정로 289'},
    {'name':'봉명역', 'addr':'천안 동남구 차돌로 51'},
    {'name':'쌍용역', 'addr':'천안 서북구 쌍용19로 20'},
]

infra_df = pd.DataFrame(infra_data)

In [6]:
# naver map api: geocode
def find_XY(addr):
    client_id = "NAVER MAP API ID"
    client_secret = "NAVER MAP API SECRET"

    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={addr}"

    # 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # 요청
    response = requests.get(url, headers=headers)
    tmp = response.json()

    try:
        long = tmp['addresses'][0]['x']
        lat = tmp['addresses'][0]['y']
    except:
        return None, None

    return lat, long

In [7]:
# 빵소
for i, addr in enumerate(pan_df['addr']):
    lat, long = find_XY(addr)
    pan_df.loc[i, 'lat'] = lat
    pan_df.loc[i, 'long'] = long

# 폐건물
for i, addr in enumerate(building_df['addr']):
    lat, long = find_XY(addr)
    building_df.loc[i, 'lat'] = lat
    building_df.loc[i, 'long'] = long

# 인프라
for i, addr in enumerate(infra_df['addr']):
    lat, long = find_XY(addr)
    infra_df.loc[i, 'lat'] = lat
    infra_df.loc[i, 'long'] = long

In [8]:
pan_df

,name,addr,lat,long
0,꼬망스케익,천안 서북구 봉정로 362,36.8343409,127.1457984
1,듀팡과자점,천안 서북구 늘푸른5길 22,36.8273293,127.1296515
2,뚜쥬루돌가마점,천안 동남구 풍세로 706,36.7730783,127.1395332
3,못난이꽈배기,천안 동남구 사직로 10-6,36.8006604,127.1500443
4,몽상가인,천안 서북구 쌍용대로 85,36.8051389,127.1318129
5,브레드보드,천안 동남구 성남면 종합휴양지로 183,36.7573144,127.2250910
6,쁘띠빠리,천안 서북구 한들3로 88,36.8310439,127.1244483
7,수제빵연구소,천안 서북구 백석3로 35,36.8218748,127.1157124
8,슈에뜨베이커리,천안 동남구 서부대로 226-12,36.7871694,127.1300561
9,시바앙과자점,천안 동남구 충절로 42,36.8151732,127.1575695


In [9]:
building_df

,name,addr,lat,long
0,두정동 폐공사장,충남 천안시 서북구 두정동 1354,36.8319745,127.1385192
1,쌍용동 폐공장,천안시 서북구 충무로 214,36.7963716,127.1293773
2,삼룡동 폐건물,천안시 동남구 삼룡동 240-3,36.7901058,127.1644011


In [10]:
infra_df

,name,addr,lat,long
0,천안역 1호선,천안 동남구 대흥로 239,36.8123208,127.1468225
1,천안아산역,충청남도 아산시 배방읍 희망로 100,36.7945269,127.1044505
2,천안고속버스터미널,천안 동남구 만남로 43,36.8207735,127.1565280
3,두정역,천안 서북구 두정로 289,36.8329347,127.1490207
4,봉명역,천안 동남구 차돌로 51,36.8012573,127.1358356
5,쌍용역,천안 서북구 쌍용19로 20,36.7937242,127.1213504


In [11]:
pan_df['grp'] = '빵소'
infra_df['grp'] = '인프라'
building_df['grp'] = '폐건물'

In [12]:
fpath = './천안_등산로/'
file_list = os.listdir(fpath)
file_list = [f for f in file_list if f.endswith('.json')]

In [13]:
def find_geometry_by_detail_spo(data):
    entrance = list()
    for feature in data['features']:
        attributes = feature.get('attributes', {})
        if attributes.get('DETAIL_SPO') in ['시종점', '등산로입구', '시점']:
            entrance.append(feature.get('geometry', {}))
    return entrance

In [14]:
file_list

['PMNTN_SPOT_매봉산_441300801.json',
 'PMNTN_SPOT_노태산_441300501.json',
 'PMNTN_SPOT_광덕산_441300301.json',
 'PMNTN_SPOT_국사봉_441304101.json',
 'PMNTN_SPOT_금북정맥_441300103.json',
 'PMNTN_SPOT_만경산_441300701.json',
 'PMNTN_SPOT_백운산_441301201.json',
 'PMNTN_SPOT_구성산_441303901.json',
 'PMNTN_SPOT_고려산_441303801.json',
 'PMNTN_SPOT_거머산_441300201.json']

In [15]:
mountain_df = pd.DataFrame()
for fname in file_list:
    with open(fpath+fname, 'r') as f:
        json_data = json.load(f)

    m_name = fname.split('_')[-2]
    geo_list = find_geometry_by_detail_spo(json_data)

    for i, m_geo in enumerate(geo_list):
        ser = pd.Series({'name':m_name+str(i+1), 'X':m_geo['x'], 'Y':m_geo['y']})
        mountain_df = pd.concat([mountain_df, ser], axis=1)

In [16]:
mountain_df = mountain_df.T.reset_index().drop(columns='index')

In [17]:
def trans_coord(X, Y):
    # 산림청 좌표계: grs80타원체의 tm 중부(20,60) -> EPSG5186
    # TMAP/NAVER API 좌표계: GPS 경위도 WGS84 -> epsg:4326
    xy = pyproj.Proj(init="epsg:5186")
    longlat = pyproj.Proj(init="epsg:4326")

    long, lat = pyproj.transform(xy, longlat, X, Y)
    return long, lat

In [18]:
mountain_df.head()

,name,X,Y
0,매봉산1,227442.58,462447.6964
1,매봉산2,227569.0004,462303.1572
2,매봉산3,227573.3753,461331.6998
3,매봉산4,227970.8156,462444.1965
4,매봉산5,228310.0351,462121.6521


In [19]:
for i, row in mountain_df.iterrows():
    long, lat = trans_coord(row.X, row.Y)
    mountain_df.at[i, 'lat'] = lat
    mountain_df.at[i, 'long'] = long

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/ky/h5cjl7dj2y9cj2mn9cv958kc0000gn/T/ipykernel_22262/37784505.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.g

In [20]:
mountain_df.head()

,name,X,Y,lat,long
0,매봉산1,227442.58,462447.6964,36.760223,127.307340
1,매봉산2,227569.0004,462303.1572,36.758917,127.308751
2,매봉산3,227573.3753,461331.6998,36.750163,127.308765
3,매봉산4,227970.8156,462444.1965,36.760176,127.313256
4,매봉산5,228310.0351,462121.6521,36.757260,127.317043


In [21]:
mountain_df['grp'] = '산'

In [ ]:
pd.concat([pan_df, building_df, infra_df, mountain_df])[['name','addr','lat','long','grp']].to_csv('./위경도.csv', index=False)